There are two primary types of transfer learning:

Transfer learning via **feature extraction**: We remove the FC layer head from the pre-trained network and replace it with a softmax classifier. This method is super simple as it allows us to treat the pre-trained CNN as a feature extractor and then pass those features through a Logistic Regression classifier.

Transfer learning via **fine-tuning**: When applying fine-tuning, we again remove the FC layer head from the pre-trained network, but this time we construct a brand new, freshly initialized FC layer head and place it on top of the original body of the network. The weights in the body of the CNN are frozen, and then we train the new layer head (typically with a very small learning rate). We may then choose to unfreeze the body of the network and train the entire network.

## Feature Extraction

In [49]:
import torch
from torchvision import models
from torchsummary import summary
from tqdm import tqdm

In [50]:
model = models.resnet50(pretrained=True)

for key, module in model._modules.items():
    print(key)    

conv1
bn1
relu
maxpool
layer1
layer2
layer3
layer4
avgpool
fc


In [59]:
summary(model, (3, 1024, 1024))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 512, 512]           9,408
       BatchNorm2d-2         [-1, 64, 512, 512]             128
              ReLU-3         [-1, 64, 512, 512]               0
         MaxPool2d-4         [-1, 64, 256, 256]               0
            Conv2d-5         [-1, 64, 256, 256]           4,096
       BatchNorm2d-6         [-1, 64, 256, 256]             128
              ReLU-7         [-1, 64, 256, 256]               0
            Conv2d-8         [-1, 64, 256, 256]          36,864
       BatchNorm2d-9         [-1, 64, 256, 256]             128
             ReLU-10         [-1, 64, 256, 256]               0
           Conv2d-11        [-1, 256, 256, 256]          16,384
      BatchNorm2d-12        [-1, 256, 256, 256]             512
           Conv2d-13        [-1, 256, 256, 256]          16,384
      BatchNorm2d-14        [-1, 256, 2

In [61]:
modelOutputFeats = model.fc.in_features
model.fc = torch.nn.Linear(modelOutputFeats, 5)
summary(model, (3, 1024, 1024))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 512, 512]           9,408
       BatchNorm2d-2         [-1, 64, 512, 512]             128
              ReLU-3         [-1, 64, 512, 512]               0
         MaxPool2d-4         [-1, 64, 256, 256]               0
            Conv2d-5         [-1, 64, 256, 256]           4,096
       BatchNorm2d-6         [-1, 64, 256, 256]             128
              ReLU-7         [-1, 64, 256, 256]               0
            Conv2d-8         [-1, 64, 256, 256]          36,864
       BatchNorm2d-9         [-1, 64, 256, 256]             128
             ReLU-10         [-1, 64, 256, 256]               0
           Conv2d-11        [-1, 256, 256, 256]          16,384
      BatchNorm2d-12        [-1, 256, 256, 256]             512
           Conv2d-13        [-1, 256, 256, 256]          16,384
      BatchNorm2d-14        [-1, 256, 2